<a href="https://colab.research.google.com/github/Param-Bhatt/NNFL-Assignment/blob/master/2/Q6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Mounting drive

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h3>Navigating to respective directory

In [15]:
%cd "/content/drive/My Drive/NNFL-Assignments/2"
!ls

/content/drive/My Drive/NNFL-Assignments/2
assignment2.gdoc  class2_images    class_label.mat  data5.mat	  input.mat
assignment2.pdf   class3_images    data55.xlsx	    input_a2.mat  label.mat
class1_images	  class_label.csv  data5.csv	    input.gsheet  Q8.ipynb


<h3>Importing all libraries required

In [16]:
import numpy as np 
from scipy.io import loadmat
from sklearn.preprocessing import normalize
import random
from sklearn.metrics import confusion_matrix as cm

<h3>Activation functions

In [17]:
#Gaussian activation function
def gaussian(X, a, b):   
    K = np.zeros((X.shape[0], hidden_neurons))
    for i in range(K.shape[0]):
        for j in range(K.shape[1]):
            K[i,j] = np.exp(-b[j] * np.linalg.norm(a[:,j] - X[i,:]))  
    return K

#Tanh activation function
def tanh(X):
    return np.tanh(X)



<h3>Initialise data

In [18]:
def init_data(data):
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = normalize(X, axis = 0)
    return X, y

f = loadmat('data5.mat')
D = f['x']
np.random.shuffle(D)

X_tot, y_tot = init_data(D)
X_tot = np.insert(X_tot, 0, 1, axis=1)


labels = D[:, -1]
y = np.zeros([len(X_tot), 2])

for i in range(len(labels)):
    if labels[i] == 1:
        y[i,1] = 1.0
    elif labels[i] == 0:
        y[i,0] = 1.0

hidden_neurons = 300
output_neurons = 2

<h3>K fold validation for gaussian activation

In [26]:
%%time

print("Gaussian accuracies: ")
acc = 0
for k in range(5):

    X_train = X_tot[0 : 1718]
    y_train = y[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y[1718 :]
    
    a = np.random.rand(X_train.shape[1], hidden_neurons) 
    b = np.random.rand(hidden_neurons)
    
    # Training 
    H = gaussian(X_train, a, b)
    H_inv = np.linalg.pinv(H)
    W2 = np.matmul(H_inv, y_train)
    
    # Testing
    H_T = gaussian(X_val, a, b)
    y_pred = np.matmul(H_T, W2)
    
    print(f'Fold {k+1}:\n')
    a = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
    b= [np.argmax(y_val[i]) for i in range(len(y_val))]
    confmat = cm(b,a)
    print("Confusion Matrix:\n")
    print(confmat)
    Accuracy = (confmat[0][0]+confmat[1][1])/len(y_val)
    acc += Accuracy
    Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    print(f"Accuracy: {Accuracy}\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}")
    print("=============================================")

    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X_train
    y[0 : 430] = y_val
    y[430 : ] = y_train

print(f"Overall Accuracy: {acc*100/5}%")

Gaussian accuracies: 
Fold 1:

Confusion Matrix:

[[198  21]
 [ 13 198]]
Accuracy: 0.9659302325581396
Sensitivity: 0.9383886255924171
Specificity: 0.9041095890410958
Fold 2:

Confusion Matrix:

[[192  26]
 [ 22 190]]
Accuracy: 0.9333720930232559
Sensitivity: 0.8962264150943396
Specificity: 0.8807339449541285
Fold 3:

Confusion Matrix:

[[191  29]
 [ 24 186]]
Accuracy: 0.9217441860465116
Sensitivity: 0.8857142857142857
Specificity: 0.8681818181818182
Fold 4:

Confusion Matrix:

[[188  27]
 [ 35 180]]
Accuracy: 0.9008139534883721
Sensitivity: 0.8372093023255814
Specificity: 0.8744186046511628
Fold 5:

Confusion Matrix:

[[189  31]
 [ 16 194]]
Accuracy: 0.9356976744186047
Sensitivity: 0.9238095238095239
Specificity: 0.8590909090909091
Overall Accuracy: 93.15116279069767%
CPU times: user 38.7 s, sys: 940 ms, total: 39.6 s
Wall time: 38.2 s


<h3>K fold validation for tanh activation

In [25]:
%%time

y_pred_full = []
print("Tanh accuracies:\n")
acc = 0
for k in range(5):
    
    X_train = X_tot[0 : 1718]
    y_train = y[0 : 1718]
    X_val = X_tot[1718 :]
    y_val = y[1718 :]
    
    a = np.random.rand(X_train.shape[1],hidden_neurons) 
    b = np.random.rand(hidden_neurons)
    
    # Training 
    H = tanh(X_train)
    H_inv = np.linalg.pinv(H)
    W2 = np.matmul(H_inv, y_train)
    
    # Testing
    H_T = tanh(X_val)
    y_pred = np.matmul(H_T, W2)
    
    print(f'Fold {k+1}:\n')
    a = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
    b= [np.argmax(y_val[i]) for i in range(len(y_val))]
    confmat = cm(b,a)
    print("Confusion Matrix:\n")
    print(confmat)
    Accuracy = (confmat[0][0]+confmat[1][1])/len(y_val)
    acc += Accuracy
    Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
    Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
    print(f"Accuracy: {Accuracy}\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}")
    print("=============================================")
    

    X_tot[0 : 430] = X_val
    X_tot[430 : ] = X_train
    y[0 : 430] = y_val
    y[430 : ] = y_train
print(f"Overall Accuracy: {acc*100/5}%")

Tanh accuracies:

Fold 1:

Confusion Matrix:

[[182  34]
 [ 24 190]]
Accuracy: 0.9541162790697674
Sensitivity: 0.8878504672897196
Specificity: 0.8425925925925926
Fold 2:

Confusion Matrix:

[[186  33]
 [ 26 185]]
Accuracy: 0.9517906976744186
Sensitivity: 0.8767772511848341
Specificity: 0.8493150684931506
Fold 3:

Confusion Matrix:

[[179  38]
 [ 38 175]]
Accuracy: 0.9122558139534883
Sensitivity: 0.8215962441314554
Specificity: 0.8248847926267281
Fold 4:

Confusion Matrix:

[[179  41]
 [ 46 164]]
Accuracy: 0.8866744186046511
Sensitivity: 0.780952380952381
Specificity: 0.8136363636363636
Fold 5:

Confusion Matrix:

[[181  34]
 [ 26 189]]
Accuracy: 0.9494651162790697
Sensitivity: 0.8790697674418605
Specificity: 0.8418604651162791
Overall Accuracy: 93.0860465116279%
CPU times: user 179 ms, sys: 78.4 ms, total: 257 ms
Wall time: 132 ms
